In [1]:
import os
import chardet
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter


def load_markdown(data_path):
    with open(data_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )

    with open(data_path, 'rt', encoding=encoding) as file:
        data_string = file.read()
        documents = markdown_splitter.split_text(data_string)

        # 파일명을 metadata에 추가
        domain = data_path  # os.path.basename(data_path)
        for doc in documents:
            if not doc.metadata:
                doc.metadata = {}
            doc.metadata["domain"] = domain  # Document 객체의 metadata 속성에 파일명 추가

        return documents


def load_txt(data_path):
    with open(data_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    text_splitter = CharacterTextSplitter(
        separator="\n",
        length_function=len,
        is_separator_regex=False,
    )

    with open(data_path, 'r', encoding=encoding) as file:
        data_string = file.read().split("\n")
        domain = data_path  # os.path.basename(data_path)
        documents = text_splitter.create_documents(data_string)

        for doc in documents:
            if not doc.metadata:
                doc.metadata = {}
            doc.metadata["domain"] = domain  # Document 객체의 metadata 속성에 파일명 추가

        return documents


def load_general(base_dir):
    data = []
    cnt = 0
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if ".txt" in file:
                cnt += 1
                data += load_txt(os.path.join(root, file))

    print(f"the number of txt files is : {cnt}")
    return data


def load_document(base_dir):
    data = []
    cnt = 0
    for root, _, files in os.walk(base_dir):
        for file in files:
            if ".md" in file:
                cnt += 1
                data += load_markdown(os.path.join(root, file))

    print(f"the number of md files is : {cnt}")
    return data



In [2]:
def get_markdown_files(source_dir):
    md_data = load_document(base_dir=source_dir)
    text_data = load_general(base_dir=source_dir)

    return md_data + text_data



In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers.abstract_query import ComparativeAbstractQuerySynthesizer
# from ragas.testset.synthesizers.base import BaseSynthesizer
# from ragas.testset.synthesizers.base_query import QuerySynthesizer
from ragas.testset.synthesizers.specific_query import SpecificQuerySynthesizer
from ragas.llms import LangchainLLMWrapper

load_dotenv()

generator_llm = ChatOpenAI(model="gpt-4o-mini")
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# critic_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
# embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
generator = TestsetGenerator.from_langchain(generator_llm)
md_files = get_markdown_files(source_dir="/Users/yechanyun/YYC/work/project/AI_Application/rag/data/exynos-ai-studio-docs-main")
test_set = generator.generate_with_langchain_docs(md_files,
                                                  testset_size=1,
                                                  query_distribution=[
                                                    (ComparativeAbstractQuerySynthesizer(llm=ragas_llm), 0.5),
                                                    (SpecificQuerySynthesizer(llm=ragas_llm), 0.5),],
                                                  with_debugging_logs=True)

In [ ]:
test_set.dict()

In [ ]:
evaluation_dataset = test_set.to_evaluation_dataset()
evaluation_dataset
print(evaluation_dataset.dict())

In [25]:
evaluation_dataset.to_jsonl("./evaluation_dataset.jsonl")

In [26]:
test_set.to_jsonl("./testset.jsonl")

In [ ]:
from tsk_ragtools import text_gen as tg
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_jsonl('./evaluation_dataset.jsonl')
evaluation_dataset_dict = evaluation_dataset.dict()

for i, obj in enumerate(evaluation_dataset_dict["samples"]):
  print(
    f"\rGenerating RAGAS json...({i+1}/{len(evaluation_dataset_dict['samples'])})",
    end="",
    flush=True,
  )
  response = tg.answer_question(obj["user_input"])
  obj.update({"retrieved_contexts": response["list"], "response": response["response"]})

print(evaluation_dataset_dict)

In [4]:
eval_dataset = EvaluationDataset.from_dict(mapping=evaluation_dataset_dict["samples"])

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI

metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]
metrics_names = [m.name for m in metrics]
print(metrics_names)

In [ ]:
import ragas.metrics as ragas_metrics

metrics_candidates = ["LLMContextRecall", "Faithfulness", "FactualCorrectness", "SemanticSimilarity"]

metrics = []

for m in metrics_candidates:
    metrics.append(getattr(ragas_metrics, m))

metrics_names = [m.name for m in metrics]
print(metrics_names)

In [ ]:
results.to_pandas().to_dict()

In [1]:
from actions.ragas_testset_creator import RagasTestsetCreator

import logging

logger = logging.getLogger(__name__)

args = {
    "action": "createTestset",
    "json_filename": ".\\testset_with_context.json",
    "csv_filename": "",
    "basepath": "",
    "gpt_model": "gpt-4o-mini",
    "dataset_source_dir": "C:\\work\\project\\AI_Application\\rag\\data\\exynos-ai-studio-docs-main",
    "testset_test_size": 2,
    "testset_comparative_query_ratio": 0.5,
    "testset_specific_query_ratio": 0.5,
    "testset_filename": "testset.json",
    "eval_result_filename": ".\\eval_result.json"
}


ragasTestsetCreator = RagasTestsetCreator(logger)
test_set, generator = ragasTestsetCreator.main(args["dataset_source_dir"],
                            args["testset_test_size"],
                            args["testset_comparative_query_ratio"],
                            args["testset_specific_query_ratio"],
                            args["gpt_model"],
                            args["testset_filename"])

print(test_set)

c:\work\project\AI_Application\rag\source\ragtools-all-in-one\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\work\project\AI_Application\rag\source\ragtools-all-in-one\Lib\site-packages\ragas\prompt\base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.llms.prompt import PromptValue


{'Header 1': 'System requirement', 'Header 2': 'Software', 'location': 'C:\\work\\project\\AI_Application\\rag\\data\\exynos-ai-studio-docs-main\\system_requirements\\system_requirements.md'}


Generating Samples: 100%|██████████| 2/2 [00:09<00:00,  4.69s/it]                                                    


EvaluationDataset(features=['eval_sample', 'synthesizer_name'], len=2)


In [6]:
test_set.samples[1]

TestsetSample(eval_sample=SingleTurnSample(user_input='What role does the NPU Simulator play in the ELT framework?', retrieved_contexts=None, reference_contexts=['ELT](elt/elt.md)\n- [Model Analyzer](elt/model_analyzer/model_analyzer.md)\n- [SCVT](elt/scvt/scvt.md)\n- [Quantizer](elt/quantizer/quantizer.md)\n- [NPU Simulator](elt/npu_simulator/npu_simulator.md)\n- [Performance Estimator](elt/performance_estimator/performance_estimator.md)\n- [Graph Generator](elt/graph_generator/graph_generator.md)\n- ['], response=None, multi_responses=None, reference='The NPU Simulator plays a role in the ELT framework, although the specific details of its function are not provided in the context.', rubric=None), synthesizer_name='SpecificQuerySynthesizer')

In [9]:
generator.knowledge_graph.nodes

[Node(id: f9e4f4, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'summary', 'headlines', 'summary_embedding', 'embedding', 'title', 'keyphrases']),
 Node(id: dd8576, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'summary', 'headlines', 'summary_embedding', 'title', 'embedding', 'keyphrases']),
 Node(id: 089a7d, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding', 'embedding', 'title', 'keyphrases']),
 Node(id: 0f397a, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding', 'embedding', 'title', 'keyphrases']),
 Node(id: 4481ee, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding', 'embedding', 'title', 'keyphrases']),
 Node(id: 9a009c, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'summary', 'headlines', 'su

In [ ]:
import json

with open(".\\eval_result_241018.json", "r") as f:
    json_data = json.load(f)

context_recall = 0
factual_correctness = 0
faithfulness = 0
semantic_similarity = 0
for i in len(json_data["context_recall"]):
    context_recall += json_data["context_recall"][i]
for i in len(json_data["factual_correctness"]):
    factual_correctness += json_data["factual_correctness"][i]
for i in len(json_data["faithfulness"]):
    faithfulness += json_data["faithfulness"]
for i in len(json_data["semantic_similarity"]):
    semantic_similarity += json_data["semantic_similarity"][i]

print(context_recall/len(json_data["context_recall"]))
print(factual_correctness/len(json_data["factual_correctness"]))
print(faithfulness/len(json_data["faithfulness"]))
print(semantic_similarity/len(json_data["semantic_similarity"]))